<table width='100%'>
<tr>
<td style='background-color:white'>
    <p align="left">
    Exercises for the course<br>
        <b>Deep Learning 1</b><br>
    Winter Semester 2022/23
    </p>
</td>
<td style='background-color:white'>
    Machine Learning Group<br>
    <b>Faculty IV – Electrical Engineering and Computer Science</b><br>
    Technische Universität Berlin
</td>
</tr>
</table>

<br>
<center>
    <h1>Exercise Sheet 1-2 (programming part)</h1>
</center>
<br>

In this homework, our goal is to test different approaches to implement neural networks. Here, we will be focusing on programming forward and backward computations. Training neural networks will be done in the next homework. The neural network we consider is depicted below:

![](files/net.svg.png)

## Part 1: Implementing of Backpropagation (10 P)

The following code implements the forward pass of this network in numpy. Here, you are asked to implement the backward pass, and obtain the gradient with respect to the weight and bias parameters.

In [118]:
import numpy as np
import utils
import torch

# 1. Get the data and parameters

X,T = utils.getdata()
W,B = utils.getparams()
A = [X]
# 2. Run the forward pass

for i in range(3): A.append(np.maximum(0,A[-1].dot(W[i])+B[i]))
Y = A[-1].dot(W[3])+B[3]

# 3. Compute the error

err = ((Y-T)**2).mean()

# 4. Error backpropagation (TODO: replace by your code)

DW4 = 2 * (Y - T) * A[-1]

DW3 = 2 * (Y - T) * W[3] @ A[-2]*(A[-1]>0)

DW2 = torch.zeros((6,6))
for i in range(6):
    for j in range(6):
        entry = 2 * (Y - T) * A[1][0][i] *(A[2][0][j]>0) * W[3].T @ (W[2][j, :] *(A[3][0]>0))
        DW2[i,j] = entry[0]

DW1 = torch.zeros((4,6))
for k in range(4):
    for l in range(6):
        entry_main = 2 * (Y - T) * A[0][0][k] *(A[1][0][l]>0) 
        sum_i = 0
        for i in range(6):
            entry_sup = 0
            for j in range(6):
                entry_sup += W[2][i,j]*(A[3][0][j]>0)*W[3][j]
            sum_i += entry_sup * W[1][l,i] * (A[3][0][i]>0)
        DW1[k][l] = (entry_main * sum_i)[0][0]



# 5. Show error gradient w.r.t. the 1st weight parameter

print(np.linalg.norm(DW[0][0,0]))

(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)
(1, 1) (1,)


NameError: name 'DW' is not defined

## Part 2: Using Automatic Differentiation (10 P)

Because gradient computation can be error-prone, we often rely on libraries that incorporate automatic differentiation. In this exercise, we make use of the PyTorch library. You are then asked to compute the error of the neural network within that framework, which will then be automatically differentiated.

In [114]:
import torch
import torch.nn as nn

# 1. Get the data and parameters

X,T = utils.getdata()
W,B = utils.getparams()

# 2. Convert to PyTorch objects

X = torch.Tensor(X)
T = torch.Tensor(T)
W = [nn.Parameter(torch.Tensor(w)) for w in W]
B = [nn.Parameter(torch.Tensor(b)) for b in B]

# 3. Compute the forward pass and the error (TODO: replace by your code)

A = [X]

for i in range(3):
    A.append(torch.maximum(torch.tensor(0), A[-1] @ W[i]+ B[i]))
    
Y = A[-1] @ W[3] + B[3]

err = ((Y-T)**2).mean()


# 4. Apply automatic differentiation

err.backward()

# 5. Show error gradient w.r.t. the 1st weight parameter

print(np.linalg.norm(W[0].grad[0,0]))

1.5422822


## Part 3: Object-Oriented Implementation (10 P)

As a last exercise, we would like to make use of existing neural network objects of the PyTorch library. Here, most of the code is already implemented for you. You are only asked to find where the error gradient of the first weight parameter has been stored, and to print it.

In [72]:
import torch
import torch.nn as nn


# 1. Get the data and parameters

X,T = utils.getdata()
W,B = utils.getparams()

# 2. Convert to PyTorch objects

X = torch.Tensor(X)
T = torch.Tensor(T)
W = [torch.nn.Parameter(torch.Tensor(w.T)) for w in W]
B = [torch.nn.Parameter(torch.Tensor(b)) for b in B]

# 3. Build the neural network

net = torch.nn.Sequential(
          nn.Linear(4,6),nn.ReLU(),
          nn.Linear(6,6),nn.ReLU(),
          nn.Linear(6,6),nn.ReLU(),
          nn.Linear(6,1))
          

for l,w,b in zip(list(net)[::2],W,B):
    l.weight = w
    l.bias = b
    
# 4. Compute the forward pass and the error gradient

Y = net.forward(X)
err = ((Y-T)**2).mean()
err.backward()

# 5. Show error gradient w.r.t. the 1st weight parameter (TODO: replace by your code)

print(f"Error w.r.t. the first weight parameter: {net[0].weight.grad[0][0]} (3 d.p.)")

Error w.r.t. the first weight parameter: -1.542282223701477 (3 d.p.)
